# Assignment 5 - Text Analysis
An explanation this assignment could be found in the .pdf explanation document


## Materials to review for this assignment
<h4>From Moodle:</h4> 
<h5><u>Review the notebooks regarding the following python topics</u>:</h5>
<div class="alert alert-info">
&#x2714; <b>Working with strings</b> (tutorial notebook)<br/>
&#x2714; <b>Text Analysis</b> (tutorial notebook)<br/>
&#x2714; <b>Hebrew text analysis tools (tokenizer, wordnet)</b> (moodle example)<br/>
&#x2714; <b>(brief review) All previous notebooks</b><br/>
</div> 
<h5><u>Review the presentations regarding the following topics</u>:</h5>
<div class="alert alert-info">
&#x2714; <b>Text Analysis</b> (lecture presentation)<br/>
&#x2714; <b>(brief review) All other presentations</b><br/>
</div>

## Preceding Step - import modules (packages)
This step is necessary in order to use external modules (packages). <br/>

In [1]:
# --------------------------------------
import pandas as pd
import numpy as np
# --------------------------------------


# --------------------------------------
# ------------- visualizations:
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
# --------------------------------------


# ---------------------------------------
import sklearn
from sklearn import preprocessing, metrics, pipeline, model_selection, feature_extraction 
from sklearn import naive_bayes, linear_model, svm, neural_network, neighbors, tree
from sklearn import decomposition, cluster

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV 
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score,make_scorer
from sklearn.metrics import mean_squared_error, r2_score, silhouette_score
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder

from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import Perceptron, SGDClassifier
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
# ---------------------------------------


# ----------------- output and visualizations: 
import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.simplefilter("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter("ignore", category=ConvergenceWarning)
# show several prints in one cell. This will allow us to condence every trick in one cell.
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
%matplotlib inline
pd.pandas.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
# ---------------------------------------

### Text analysis and String manipulation imports:

In [2]:
# --------------------------------------
# --------- Text analysis and Hebrew text analysis imports:
# vectorizers:
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# regular expressions:
import re
# --------------------------------------

### (optional) Hebrew text analysis - WordNet (for Hebrew)
Note: the WordNet is not a must

#### (optional) Only if you didn't install Wordnet (for Hebrew) use:

In [3]:
# word net installation:

# unmark if you want to use and need to install
# !pip install wn
# !python -m wn download omw-he:1.4

In [4]:
# word net import:

# unmark if you want to use:
import wn

### (optional) Hebrew text analysis - hebrew_tokenizer (Tokenizer for Hebrew)
Note: the hebrew_tokenizer is not a must

#### (optional) Only if you didn't install hebrew_tokenizer use:

In [5]:
# Hebrew tokenizer installation:

# unmark if you want to use and need to install:
# !pip install hebrew_tokenizer

In [6]:
# Hebrew tokenizer import:

# unmark if you want to use:
import hebrew_tokenizer as ht

C:\Users\USER\Desktop\Machine Learning\Assignment5-text-analysis


### Reading input files
Reading input files for train annotated corpus (raw text data) corpus and for the test corpus

In [7]:
train_filename = 'annotated_corpus_for_train.csv'
test_filename  = 'corpus_for_test.csv'
df_train = pd.read_csv(train_filename, index_col=None, encoding='utf-8')
df_test  = pd.read_csv(test_filename, index_col=None, encoding='utf-8')

In [8]:
df_train.head(8)
df_train.shape

,story,gender
0,"כשחבר הזמין אותי לחול, לא באמת חשבתי שזה יקרה,...",m
1,לפני שהתגייסתי לצבא עשיתי כל מני מיונים ליחידו...,m
2,מאז שהתחילו הלימודים חלומו של כל סטודנט זה הפנ...,f
3,"כשהייתי ילד, מטוסים היה הדבר שהכי ריתק אותי. ב...",m
4,‏הייתי מדריכה בכפר נוער ומתאם הכפר היינו צריכי...,f
5,לפני כ3 חודשים טסתי לרומא למשך שבוע. טסתי במטו...,f
6,אני כבר שנתיים נשוי והשנה אני ואישתי סוף סוף י...,m
7,השנה התחלנו שיפוץ בדירה שלנו בתל אביב. הדירה ה...,f


(753, 2)

In [9]:
df_test.head(3)
df_test.shape

,test_example_id,story
0,0,כל קיץ אני והמשפחה נוסעים לארצות הברית לוס אנג...
1,1,"הגעתי לשירות המדינה אחרי שנתיים כפעיל בתנועת ""..."
2,2,אחת האהבות הגדולות שלי אלו הכלבים שלי ושל אישת...


(323, 2)

## Your implementation:
Write your code solution in the following code-cells

### Pre-Processing:

In [10]:
# Over-ride tokenizer
wordnet_he = wn.Wordnet('omw-he:1.4')

def tokenize(text):
    t_list = list()
    tokens = ht.tokenize(text)
    
    for grp, token, token_num, (start_index,end_index) in tokens:
        token = token.lstrip('ו')
        
        synsets = wordnet_he.synsets(token)
        if (len(synsets) > 0):
            t_list.append(synsets[0].lemmas()[0])
        else:
            t_list.append(token)
    return t_list

In [11]:
#clean the text

def clean(df):
    for row in df.index:
        df['story'][row] = re.sub(r'[^\w\s]','',df['story'][row]) # remove punctuation
        df['story'][row] = re.sub(r'\b\w+[a-zA-Z]\w*\b', '', df['story'][row]) # remove english words
        df['story'][row] = re.sub(r'\d+','',df['story'][row]) # remove digits
    return df

In [12]:
# change the y label to numeric
# implmeantion the clean function on the df_train and df_test

df_train['gender'] = df_train['gender'].replace('f',0) # change to numeric
df_train['gender'] = df_train['gender'].replace('m',1) # change to numeric
df_test = clean(df_test)
df_train = clean(df_train)

In [13]:
# split the df_train to x_train and y_train

x_train = df_train['story']
y_train = df_train['gender']

#### Vectorizer functions

In [14]:
def tfidf(x_train):
    vec = TfidfVectorizer(tokenizer=tokenize,min_df = 3,max_features=2500,norm='l2')
    
    train = vec.fit_transform(x_train)
    return train

In [15]:
def countvec(x_train):
    vec = CountVectorizer(tokenizer=tokenize,min_df = 3,max_features=2500)

    train = vec.fit_transform(x_train)
    return train

#### Best-hyperparameters function

In [16]:
def best_params(clf, clf_params, x_train, y_train):
    grid_search = GridSearchCV(clf, clf_params, cv=10)
    grid_search.fit(x_train, y_train)
    
    print("The model is: ", clf)
    print("Best hyperparameters: ", grid_search.best_params_)
    return grid_search.best_params_

#### Evaluate function

In [17]:
def evaluate(model, x_train, y_train):
    model = model.fit(x_train,y_train)
    scores = cross_val_score(model, x_train, y_train, cv=10,scoring=make_scorer(f1_score,average='macro'))
        
    avg_f1_score = np.mean(scores)
    
    print("The average f1 is: ", avg_f1_score)

In [18]:
# Implementation of the vectorizer functions:

x_train_tf = tfidf(x_train)
x_train_cv = countvec(x_train)

###### Find the best model & vectorizer

In [19]:
## KNN

knn = KNeighborsClassifier()

knn_params = {
    'n_neighbors': [3, 5, 7],
    'weights': ['uniform', 'distance'],
    'p': [1, 2]
}

print("TfidfVectorizer")

params_tf = best_params(knn, knn_params, x_train_tf, y_train)
knn_tf = KNeighborsClassifier(n_neighbors=params_tf['n_neighbors'],
                              weights=params_tf['weights'],
                              p=params_tf['p'])

evaluate(knn_tf, x_train_tf, y_train)
print()
print("CountVectorizer")

params_cv = best_params(knn, knn_params, x_train_cv, y_train)

knn_cv = KNeighborsClassifier(n_neighbors=params_cv['n_neighbors'],
                              weights=params_cv['weights'],
                              p=params_cv['p'])

evaluate(knn_cv, x_train_cv, y_train)

TfidfVectorizer
The model is:  KNeighborsClassifier()
Best hyperparameters:  {'n_neighbors': 7, 'p': 2, 'weights': 'distance'}
The average f1 is:  0.5967218984599029

CountVectorizer
The model is:  KNeighborsClassifier()
Best hyperparameters:  {'n_neighbors': 7, 'p': 1, 'weights': 'distance'}
The average f1 is:  0.4779559435142053


In [20]:
## DecisionTree

dt= DecisionTreeClassifier()

dt_params ={
    'max_depth': [None, 2, 3, 4, 5],
    'min_samples_split': [2, 3, 4],
    'min_samples_leaf': [None, 1, 2, 3]
}

print("TfidfVectorizer")

params_tf = best_params(dt, dt_params, x_train_tf, y_train)

dt_tf = DecisionTreeClassifier(max_depth=params_tf['max_depth'],
                            min_samples_split=params_tf['min_samples_split'],
                            min_samples_leaf=params_tf['min_samples_leaf'])

evaluate(dt_tf, x_train_tf, y_train)
print()
print("CountVectorizer")

params_cv = best_params(dt, dt_params, x_train_cv, y_train)

dt_cv = DecisionTreeClassifier(max_depth=params_cv['max_depth'],
                            min_samples_split=params_cv['min_samples_split'],
                            min_samples_leaf=params_cv['min_samples_leaf'])

evaluate(dt_cv, x_train_cv, y_train)

TfidfVectorizer
The model is:  DecisionTreeClassifier()
Best hyperparameters:  {'max_depth': 4, 'min_samples_leaf': 3, 'min_samples_split': 3}
The average f1 is:  0.5437466319180457

CountVectorizer
The model is:  DecisionTreeClassifier()
Best hyperparameters:  {'max_depth': 3, 'min_samples_leaf': 2, 'min_samples_split': 2}
The average f1 is:  0.5535161911681612


In [21]:
## Perceptron

per = Perceptron()

per_params = {
    'penalty' : ['l2', 'l1', 'elasticnet'],
    'alpha' : [0.1,0.001,0.0001],
    'max_iter' : [200, 500, 1000],
    'tol' : [1e-1, 1e-2, 1e-3],
    'random_state' : [0, 42]
}
print("TfidfVectorizer")
params_tf = best_params(per, per_params, x_train_tf, y_train)

per_tf = Perceptron(penalty=params_tf['penalty'],
                    alpha=params_tf['alpha'],
                    max_iter=params_tf['max_iter'],
                    tol=params_tf['tol'],
                    random_state=params_tf['random_state'])

evaluate(per_tf, x_train_tf, y_train)
print()
print("CountVectorizer")

params_cv = best_params(per, per_params, x_train_cv, y_train)

per_cv = Perceptron(penalty=params_cv['penalty'],
                    alpha=params_cv['alpha'],
                    max_iter=params_cv['max_iter'],
                    tol=params_cv['tol'],
                    random_state=params_cv['random_state'])
    
evaluate(per_cv, x_train_cv, y_train)

TfidfVectorizer
The model is:  Perceptron()
Best hyperparameters:  {'alpha': 0.0001, 'max_iter': 200, 'penalty': 'elasticnet', 'random_state': 42, 'tol': 0.1}
The average f1 is:  0.6481998521877228

CountVectorizer
The model is:  Perceptron()
Best hyperparameters:  {'alpha': 0.0001, 'max_iter': 200, 'penalty': 'l1', 'random_state': 42, 'tol': 0.01}
The average f1 is:  0.6787931045934495


In [22]:
## LinearSVC
svc = LinearSVC()

svc_params = {
    'penalty' : ['l2', 'l1'],
    'loss' : ['hinge', 'squared_hinge'],
    'tol' : [1e-2, 1e-3, 1e-4]
}

print("TfidfVectorizer")

params_tf = best_params(svc, svc_params, x_train_tf, y_train)

svc_tf = LinearSVC(penalty=params_tf['penalty'],
                   loss=params_tf['loss'],
                   tol=params_tf['tol'])

evaluate(svc_tf, x_train_tf, y_train)
print()
print("CountVectorizer")

params_cv = best_params(svc, svc_params, x_train_cv, y_train)

svc_cv = LinearSVC(penalty=params_cv['penalty'],
                   loss=params_cv['loss'],
                   tol=params_cv['tol'])

evaluate(svc_cv, x_train_cv, y_train)

TfidfVectorizer
The model is:  LinearSVC()
Best hyperparameters:  {'loss': 'squared_hinge', 'penalty': 'l2', 'tol': 0.01}
The average f1 is:  0.6445529220498789

CountVectorizer
The model is:  LinearSVC()
Best hyperparameters:  {'loss': 'squared_hinge', 'penalty': 'l2', 'tol': 0.01}
The average f1 is:  0.6708674661555024


In [25]:
## SGD

sgd = SGDClassifier()

sgd_params = {
    'loss' : ['hinge', 'log_loss', 'perceptron'],
    'penalty' : ['l2', 'l1', 'elasticnet'],
    'tol' : [1e-2, 1e-3, 1e-4],
    'n_iter_no_change' : [3, 5, 7],
    'max_iter' : [20, 40, 50]
}

print("TfidfVectorizer")

params_tf = best_params(sgd, sgd_params, x_train_tf, y_train)

sgd_tf = SGDClassifier(loss=params_tf['loss'],
                    penalty=params_tf['penalty'],
                    tol=params_tf['tol'],
                    n_iter_no_change=params_tf['n_iter_no_change'],
                    max_iter=params_tf['max_iter'])

evaluate(sgd_tf, x_train_tf, y_train)
print()
print("CountVectorizer")

params_cv = best_params(sgd, sgd_params, x_train_cv, y_train)

sgd_cv = SGDClassifier(loss=params_cv['loss'],
                    penalty=params_cv['penalty'],
                    tol=params_cv['tol'],
                    n_iter_no_change=params_cv['n_iter_no_change'],
                    max_iter=params_cv['max_iter'])

evaluate(sgd_cv, x_train_cv, y_train)

TfidfVectorizer
The model is:  SGDClassifier()
Best hyperparameters:  {'loss': 'hinge', 'max_iter': 50, 'n_iter_no_change': 7, 'penalty': 'l1', 'tol': 0.0001}
The average f1 is:  0.7158853977039724

CountVectorizer
The model is:  SGDClassifier()
Best hyperparameters:  {'loss': 'perceptron', 'max_iter': 20, 'n_iter_no_change': 3, 'penalty': 'elasticnet', 'tol': 0.0001}
The average f1 is:  0.6647268308945019


### Predict 

In [27]:
x_test = tfidf(df_test['story'])

In [28]:
sgd_tf.fit(x_train_tf,y_train)

y_pred = sgd_tf.predict(x_test)
y_pred = np.where(y_pred == 1, 'm', 'f')

pd.DataFrame({'id_test': df_test["test_example_id"], 'y_pred': y_pred}).head()
pd.DataFrame({'id_test': df_test["test_example_id"], 'y_pred': y_pred}).tail()

SGDClassifier(max_iter=50, n_iter_no_change=7, penalty='l1', tol=0.0001)

,id_test,y_pred
0,0,m
1,1,m
2,2,f
3,3,m
4,4,f


,id_test,y_pred
318,318,m
319,319,m
320,320,m
321,321,m
322,322,m


In [29]:
# create dataframe with the test results
df_predict = pd.DataFrame(
    {'test_example_id': df_test['test_example_id'], 'predicted_category': y_pred})
df_predict

,test_example_id,predicted_category
0,0,m
1,1,m
2,2,f
3,3,m
4,4,f
...,...,...
318,318,m
319,319,m
320,320,m
321,321,m


## Save output to csv (optional)
After you're done save your output to the 'classification_results.csv' csv file.<br/>
We assume that the dataframe with your results contain the following columns:
* column 1 (left column): 'test_example_id'  - the same id associated to each of the test stories to be predicted.
* column 2 (right column): 'predicted_category' - the predicted gender value for each of the associated story. 

Assuming your predicted values are in the `df_predicted` dataframe, you should save you're results as following:

In [30]:
df_predict.to_csv('classification_results.csv',index=False)